<p float="left">
  <img style="float: right;" src="./imgs/logo1.png" width="100" />
  <img style="float: left;" src="./imgs/ipa.png" width="300" /> 
</p>

<img src="./imgs/div.png" width="100%" /> 

 
# 4. Active Learning
4.1 <span style="color:#179c80"><b>Einführung</b></span> in Active Learning <br>
4.2 <span style="color:#179c80"><b>Visualisierung</b></span> und <span style="color:#179c80"><b>Experimente</b></span><br>
4.3 Active Learning vs. <span style="color:#179c80"><b>Bayes'sche Optimierung</b></span><br>


## 4.1 Einführung
Es gibt Situationen, in denen Daten im <span style="color:#db8642">Überfluss</span> vorhanden sind, das manuelle <span style="color:#179c80"><b>Labeling</b></span> aber sehr <span style="color:#db8642">zeitaufwendig</span> ist. Dies ist zum Beispiel im Bereich der Bildsegmentierung der Fall, wo pixelgenaue Kanten eingezeichnet werden müssen. In einem solchen Szenario bietet sich die Verwendung von <span style="color:#179c80"><b>Active Learning</b></span> besonders an, um keine <span style="color:#db8642">unnötigen</span> Trainingsdaten zu erstellen - unnötig meint hier die Tatsache, dass das Modell für eine Region im Eingaberaum bereits eine <span style="color:#179c80"><b>zufriedenstellende</b></span> Performance aufweist. Typischerweise startet man mit einem recht kleinen Trainingsdatensatz und lässt fortlaufend den Algorithmus, basierend auf der sogenannten <span style="color:#179c80"><b>Aqcuisition Funktion</b></span>, die <span style="color:#179c80"><b>nächsten</b></span>  zu beschriftenden Daten <span style="color:#db8642">auswählen</span>.

  <div class="alert alert-block alert-info" style="border-color:white">
    <b>Aufgabe</b>: Welche Metrik erscheint Ihnen als Wahl einer <em>Acquisition Funktion</em> sinnvoll? Erklären Sie warum und nennen Sie mindestens zwei Möglichkeiten um diese zu berechnen.
</div>

> <b>Antwort</b>: ...

###### <div class="alert alert-block alert-success" style="background-color:#b6dad2;color:#179c80;border-color:white"> <b>Lösung</b> <small style="color:#179c80">(Klicken auf den Pfeil um die Lösung aufzuklappen)</small></span></div>

> Intuitiv sollten Daten aus Bereichen ausgewählt werden, in denen ein Modell eine <span style="color:#db8642">unsichere</span> Vorhersage macht. Die relevante Metrik ist hier also die <span style="color:#179c80"><b>Unsicherheitsschätzung</b></span>. Je nach Aufgabentyp kann diese auf unterschiedliche Art und Weise berechnet werden. Beispiele sind  die Verwendung
 <ul><li>der Varianz </li><li>der Entropie </li><li>der Mutual Information </li><li>des Variation Ratios </li><li>... </li></ul>

#####  
<img src="./imgs/div.png" width="100%" /> 

## 4.2 Visualisierung und Experimente
Mithilfe der folgenden grafischen Benutzeroberfläche können Sie selbst in die <span style="color:#db8642">Rolle</span> des Algorithmus schlüpfen und <span style="color:#179c80"><b>entscheiden</b></span>, welche Daten nacheinander für das Training eines Modells verwendet werden sollen. Das  der Aufgabe zugrunde liegende Modell ist ein <span style="color:#179c80"><b>Gauß-Prozess</b></span> (GP), da dieser mit sehr <span style="color:#db8642">wenig</span> Datenpunkte schon sehr gute Ergebnisse erzielt. Ein tiefers Verständnis über den Gauß Prozess ist  <span style="color:#db8642">nicht</span> notwendig für diese Aufgabe - betrachten Sie diesen einfach als eine <span style="color:#db8642">weitere</span> Möglichkeit, zusätzlich zur Funktionschätzung eine <span style="color:#179c80"><b>Unsicherheitsangabe</b></span> zu berechnen. Für diejenigen, die dennoch tiefer in die Theorie des GPs einsteigen wollen, empfehlen wir <a href="http://www.gaussianprocess.org/gpml/chapters/RW.pdf">Gaussian Processes for Machine Learning</a>. 

  <div class="alert alert-block alert-info" style="border-color:white">
    <b>Aufgabe</b>: Versuchen Sie mit möglichst wenig Datenpunkten einen kleinen <em>MSE</em> zu erreichen.  Beschreiben Sie dabei Ihre Vorgehensweise.
</div>

In [1]:
# Starten der GUI [Keine Änderung notwendig]
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sbn
# Imports
import numpy as np
from helper.aux_function import create_data, get_toggle_value
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from helper.plots import *
import ipywidgets as widgets
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import clear_output
import ipywidgets
from ipywidgets import HBox, VBox, RadioButtons, Layout
import matplotlib.pyplot as plt
import seaborn as sbn
from scipy.stats import beta, binom, norm
import numpy as np
from ipywidgets import *
from helper.debounce import debounce
import numpy as np
import ipywidgets as widgets
from IPython.display import clear_output, display

import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib inline
import math
import jdc
import tqdm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
class AL_Handler():
    def __init__(self):
        self.training_points = []
        self.fun = lambda x: x*np.sin(x)
        self.x = np.atleast_2d(np.linspace(0, 10, 500)).T
        self.y = self.fun(self.x)
        self.candidate = slide_x.value
        self.draw_plot()
        self.y_pred, self.sigma = None, None
        
    @debounce(0.05)
    def draw_plot(self, val=None):
        try:
            self.candidate = val["new"]
        except:
            pass
        with out:
            sbn.set()
            self.fig, self.ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 6))
            self.ax.plot(self.x, self.y, "#000000",linestyle="--", label="Zielfunktion (Ground Truth)", lw=1)
            
            self.ax.set_ylim(-10, 20)
            self.ax.plot(self.training_points, self.fun(self.training_points),"#000000", marker="x", mew=2,
                         label="Daten", linestyle="none")
            self.ax.axvline(x=self.candidate, color="#db8642", linestyle="--", lw=2)
            
            if self.y_pred is not None:
                self.ax.plot(self.x, self.y_pred, color="#179c80",lw=2, label="Vorhersage")
                self.ax.fill(
                    np.concatenate([self.x, self.x[::-1]]),
                    np.concatenate([self.y_pred - 1.9600 * self.sigma, (self.y_pred + 1.9600 * self.sigma)[::-1]]),
                    alpha=0.1,
                    color="#179c80",
                    ec="None",
                    label="Unsicherheit",
                )
            
            self.ax.legend(loc="upper left")
            clear_output(wait=True)
            plt.show(self.fig)
    
    def add_point(self, btn):
        self.training_points.append(self.candidate)
        X = np.atleast_2d( sorted(self.training_points)).T
        # Instantiate a Gaussian Process model
        np.random.seed(2)
        kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
        gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
        # Observations
        y = self.fun(X).ravel()
        gp.fit(X, y)
        # Make the prediction on the meshed x-axis (ask for MSE as well)
        self.y_pred, self.sigma = gp.predict(self.x, return_std=True)
        label1.value = f"Anzahl Daten: {len(self.training_points)}"
        label2.value = f"MSE: {'%.3f'%((self.y_pred.flatten()-self.y.flatten())**2).mean()}"
        label3.value = "Daten: "+",".join([str(xx) for xx in self.training_points])
        self.draw_plot()
    
    def reset(self, btn=None):
        self.training_points = []
        self.y_pred, self.sigma = None, None
        self.draw_plot()
        label1.value="Anzahl Daten: 0"
        label2.value="MSE: nan"
        label3.value="Daten: []"
    
    def change_fun(self, val):
        val = val["new"]
        if val == "x*sin(x)":
            self.fun = lambda x: x*np.sin(x)
        elif val == "x*sin(x)*cos(x)":
            self.fun = lambda x: x*np.sin(x)*np.cos(x)
        self.reset()
        self.y = self.fun(self.x)
        self.draw_plot()
# GUI
slide_x = widgets.FloatSlider(value=0, min=0, max=10, step=0.1,
                               layout=Layout(width="350px"),
                               description='Neuer Datenpunkt: ', width=80,
    style= {'description_width': 'initial'})
btn = widgets.Button(description="Hinzufügen")
btn2 = widgets.Button(description="Reset")
label1 = widgets.Label(value="Anzahl Daten: 0")
label2 = widgets.Label(value="MSE: nan")
label3 = widgets.Label(value="Daten: []")
toggle = widgets.ToggleButtons(
    options=["x*sin(x)", "x*sin(x)*cos(x)"],
    description='Funktion',
    layout = Layout(width="400px"),
    disabled=False,
    button_style='',
    style= {'description_width': 'initial'}
)
out = widgets.Output()
al_handler = AL_Handler()
slide_x.observe(al_handler.draw_plot, names="value")
btn.on_click(al_handler.add_point)
btn2.on_click(al_handler.reset)
toggle.observe(al_handler.change_fun, names="value")
widgets.HBox([widgets.VBox([toggle, slide_x, widgets.HBox([btn, btn2]), label1, label2, label3]), out])
        

> <b>Ihre Vorgehensweise</b>: ...

#####  
<img src="./imgs/div.png" width="100%" /> 

## 4.2 Active Learning vs. Bayes'sche Optimierung

Im oben gezeigten Beispiel fließt nur die <span style="color:#179c80"><b>Unsicherheitsberechnung</b></span> des GPs in die Acquisition Funktion ein - das muss aber nicht so sein. Es ist Sache des Entwicklers <span style="color:#db8642">zu entscheiden</span>, wie er die <span style="color:#179c80"><b>Acquisition Funktion</b></span> definiert und damit die Wahl der zu evaluierenden Samples beeinflusst.

<br>
Häufig finden wir im Maschinellen Lernen eine <span style="color:#179c80"><b>Black-Box</b></span> vor, deren 
Auswertung sich auf eine Stichprobe an einem Punkt beschränkt. Ist diese Auswertung, also die Berechnung der Modellvorhersage, <span style="color:#179c80"><b>schnell</b></span> durchzuführen, kann man an <span style="color:#db8642">vielen</span> Punkten Stichproben nehmen, z. B. durch <span style="color:#db8642">Gitter-</span> oder <span style="color:#db8642">Zufallssuche</span> und die resultierende Performance vergleichen. Ist die Bewertung einer Funktion jedoch <span style="color:#179c80"><b>teuer</b></span>, z. B. bei der Abstimmung von <span style="color:#db8642">Hyperparametern</span> eines tiefen neuronalen Netzes, dann ist es wichtig, die Anzahl der aus der Blackbox-Funktion gezogenen Stichproben zu <span style="color:#179c80"><b>minimieren</b></span>. Dies ist der Bereich, in dem <span style="color:#179c80"><b>Bayes'sche Optimierungstechniken</b></span> am nützlichsten sind. Sie versuchen, das <span style="color:#179c80"><b>globale</b></span> Optimum in einer <span style="color:#db8642">minimalen</span> Anzahl von Schritten zu finden. Auch hierfür wird eine Acquisition Funktion verwendet, um ausschließlich <span style="color:#db8642">vielversprechende</span> Parameter zu evaluieren. 
<br>

Nun etwas konkreter: Stellen Sie sich vor, Sie sind im Besitz eines Modells und versuchen den optimalen Wert für einen Prameter zu finden, um eine <span style="color:#179c80"><b>Zielfunktion</b></span> zu <span style="color:#db8642">maximieren</span> (oder minimieren). Hierfür wählen Sie einige Parameterwerte aus und evaluieren die Zielfunktion. Basierend auf diesen Stellen, welcher Parameterwert soll als nächstes ausprobiert werden? Hierfür wird auf der Basis der bisherigen Evaluierung ein ein sogennantes <span style="color:#179c80"><b>Surrogate Modell</b></span> gebildet, welches die <span style="color:#db8642">Zielfunktion</span> annähern soll. Jäufig wird hierfür ein <span style="color:#179c80"><b>Gauß Prozess</b></span> gewählt. Klassischerweiße gibt es nun zwei Metriken, welche die Wahl der nächsten Evaluierung beeinflussen:
- <span style="color:#179c80"><b>Exploration</b></span>: ein Parameterwert sollte für die nächste Evaluierung in Betracht gezogen werden, wenn die entsprechende <span style="color:#db8642">Unsicherheitsschätzung</span> des Surrogate Modells an dieser Stelle <span style="color:#db8642">hoch</span> ist - an dieser Stelle wissen wir nämlich noch nicht den genauen Funktionswert unserer Zielfunktion
- <span style="color:#179c80"><b>Exploitation</b></span>: ein Parameterwert sollte für die nächste Evaluierung in Betracht gezogen werden, wenn die entsprechende <span style="color:#db8642">Funktionsschätzung</span> des Surrogate Modells dem <span style="color:#db8642">Optimum</span> unserer Zielfunktion entspricht (also entweder sehr hoch oder niedrig ist) - an dieser Stelle erwarten wir dementsprechend eine gute Performance unseres Modells.

Häufig gewichten Acquisition Funktionen in der Bayes'schen Optimierung diese beiden Metriken durch die Berechnung des  <span style="color:#179c80"><b>Expected Improvement</b></span>s. Abschließend folgender Ausschnitt einer Optimierung:

<img src="./imgs/v1.png" width=1000>

Zum gezeigten Zeitpunkt ist die Zielfunktion (schwarz) an <span style="color:#db8642">drei</span> Stellen ausgewertet. Auf dieser Basis wird ein GP berechnet (grün). Die nächste Evaluierung der Zielfunktion findet in Iteration drei mit einem Parametewert von $\approx 0.4$ statt - hier ist sowohl die Mittelwerts- als auch Unsicherheitsschätzung des GPs <span style="color:#db8642">hoch</span> - ein vielversprechender Punkt.



Im späteren Optimierungsverlauf ergibt sich dann z.B. folgendes Bild:
<img src="./imgs/v2.png" width=1000>

Zwar ist die Vorhersage des GPs für den Bereich $1.5-2.0$ noch sehr ungenau, allerdings ist hier die Funktionsschätzung im Vergleich zu anderen Stellen recht <span style="color:#db8642">klein</span>. In diesem Fall überwiegt bei der Wahl des nächsten Parameterwerts die hohe <span style="color:#db8642">Mittelwertsvorhersage</span> für einen Wert um die $-0.5$. Dennoch wird der Bereich bei $1.5$ nicht komplett außer Acht gelassen, was am Anstieg der Acquisition Funktion beobachtet werden kann.